# Context View Out Notebook

## 0. Initialization

### 0.1. Importing libraries and setting global variables

In [7]:
import EN_17037_Recipes as enr 
import pyvista as pv
import os
import pandas as pd
import numpy as np
import trimesh as tm
import itertools
import matplotlib.pyplot as plt
import pyrebase
import json

cwd = os.getcwd()
src = os.path.dirname(cwd)

with open("fb_auth.json") as json_file:
    config = json.load(json_file)

# create firebase app
firebase = pyrebase.initialize_app(config)

# access the database
fb_db = firebase.database()

main_key = '-MXlw8J5f7I_cb9m1Am1'
# project name
project_name = 'EN_17037_context_view_' + '1_10'

In [8]:
# retrive all the data
db = fb_db.child(main_key).get()

# retrieve the data as dictionary
db_data = db.val()

# dataframes
V_df = pd.DataFrame.from_dict(db_data["V"], orient='columns')
F_df = pd.DataFrame.from_dict(db_data["F"], orient='columns')
RS_df = pd.DataFrame.from_dict(db_data["RS"], orient='columns')
RD_df = pd.DataFrame.from_dict(db_data["RD"], orient='columns')

# project name
project_name = main_key[1:]

In [9]:
# setting the rays and points
rays = [[0,0,1]]
pnts = [[-35.781,39.450,10.839]]

# loading all meshes and converting them into hb surfaces
hb_surfaces = []
# mesh = pv.read(os.path.join(src, "data", "ref_room_walls.obj"))
# hb_surf = enr.pv_mesh_to_hbsurface(mesh, 0, "wall", enr.material_plastic)
mesh_tm = tm.load_mesh(os.path.join(src, "data", "ref_room_walls.obj"))
hb_surf = enr.mesh_to_hbsurface(mesh_tm.vertices, mesh_tm.faces, 0, "that_mesh", enr.material_plastic)
hb_surfaces.extend(hb_surf)

faces have mixed data, using slow fallback!


In [10]:
analysis_grid = enr.AnalysisGrid.from_points_and_vectors(pnts, rays, 'test_grid')

# put the recipe together
rp = enr.ContextViewGridBased(analysis_grids=(analysis_grid,),
               hb_objects=hb_surfaces)
               
# write simulation to folder
batch_file = rp.write(
    target_folder='.', project_name=project_name)

Found 224 opaque surfaces.
Found 0 fenestration surfaces.
Found 0 window-groups.
Writing recipe contents to: ./MXlw8J5f7I_cb9m1Am1/gridbased


In [11]:
# run the simulation
rp.run(batch_file, debug=False)

True

In [12]:
# load rtrace results
rs_path = os.path.join(project_name, 'gridbased', 'rtrace_res.txt')
rtrace_res = pd.read_csv(rs_path, skiprows=8, sep='\t', usecols=[3,4,5], header=None, names=['int_name', 'first_dist', 'last_dist'])
rtrace_res

,int_name,first_dist,last_dist
0,that_mesh,1.261,1.261


In [ ]:
mesh = tm.Trimesh(V_df.to_numpy(),F_df.to_numpy())

In [ ]:
# convert trimesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)


# adding the base mesh: light blue
p.add_mesh(tri_to_pv(mesh), color='#abd8ff', opacity=0.5)

p.add_mesh(ray_src)
p.add_mesh(ray_src + ray_dir)
# plotting
p.show(use_ipyvtk=True)

### 0.2. Setting global variables

In [ ]:
point_grid_height = "1_7"
project_name = 'EN_17037_context_view_' + point_grid_height
# create the material dictionary
material_dict = {
    "material_plastic": enr.material_plastic,
    "material_glass": enr.material_glass
}

# load the surface information table
surface_df = pd.read_csv("cv_file_table.csv")
surface_df

### 0.3. Prepare Analysis Information

In [ ]:
# homogenous discretization of sphere
icosphere = tm.creation.icosphere(subdivisions=4, radius=1.0)
v = icosphere.vertices

# setting the rays and points
rays = [tuple(v) for v in icosphere.vertices]
grid_path = os.path.join(src, "data", "point_grid_view_" + point_grid_height + ".csv")
grid_array = np.genfromtxt(grid_path, delimiter=",")
pnts = [tuple(pnt) for pnt in grid_array]

# generate grid of test points
test_inds = [j for j in range(len(pnts)) for i in range(len(rays))]
test_pnts = [pnt for pnt in pnts for i in range(len(rays))]
test_rays = rays * len(pnts)

## 1. Simulation

### 1.1. Importing geometries and assigning materials

In [ ]:
# loading all meshes and converting them into hb surfaces
hb_surfaces = []
for _, srf in surface_df.iterrows():
    filepath = os.path.join(src, "data", srf["file_path"])
    mesh = pv.read(filepath)
    hb_surf = enr.pv_mesh_to_hbsurface(mesh, srf["surface_type"], srf["surface_name"], material_dict[srf["material_name"]])
    hb_surfaces.extend(hb_surf)

### 1.2. Create Recipe

In [ ]:
analysis_grid = enr.AnalysisGrid.from_points_and_vectors(test_pnts, test_rays, 'test_grid')

# put the recipe together
rp = enr.ContextViewGridBased(analysis_grids=(analysis_grid,),
               hb_objects=hb_surfaces)

### 1.3. Create Batch Files

In [ ]:
# write simulation to folder
batch_file = rp.write(
    target_folder='.', project_name=project_name)

### 1.4. Run the simulation

In [ ]:
# run the simulation
rp.run(batch_file, debug=False)

## 2. Results

### 2.1. Load and parse the results

In [ ]:
# load grid data
grid_df = pd.read_csv(grid_path, skiprows=0, sep=',', header=None, names=['Sx','Sy','Sz'])

# load rtrace results
rs_path = os.path.join(project_name, 'gridbased', 'rtrace_res.txt')
rtrace_res = pd.read_csv(rs_path, skiprows=8, sep='\t', usecols=[3,4,5], header=None, names=['last_int_name', 'first_int_dist', 'last_int_dist'])
rtrace_res.replace("*", "sky", inplace=True)
# load the rays
rays_path = os.path.join(project_name, 'gridbased', project_name + '.pts')
rays_df = pd.read_csv(rays_path, skiprows=0, sep=' ', header=None, names=['Sx','Sy','Sz','Rx', 'Ry', 'Rz'])
print(rays_df.shape)
# concatenate data frames and group by point ids
grid_results = pd.concat([rays_df, rtrace_res], axis = 1)

# create masks
grid_results['window_mask'] = grid_results['last_int_name'] != "wall"
grid_results['horizon_mask'] = grid_results['Rz'].abs() < 0.01
grid_results['hw_mask'] = grid_results['window_mask'] * grid_results['horizon_mask']

#create groupby
gb = grid_results.groupby(test_inds)

### 2.2. Compute window view solid angle on each grid point

In [ ]:
# window view solid angle in steradian
grid_df["window_solid_angle"] = 4 * np.pi * gb['window_mask'].sum() / len(rays)
grid_df

In [ ]:
fig = plt.figure(figsize = (6,8))
plt.style.use(['ggplot'])
plt.set_cmap('viridis')
# fig.patch.set_facecolor('white')
plt.imshow(grid_df["window_solid_angle"].to_numpy().reshape((-1, 25)), interpolation='none')

plt.axis('off')
cbar = plt.colorbar() 
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Window Solid Angle (Steradian)', rotation=270, ha='center', va='center')  

plot_path = os.path.join('plots', project_name + '_window_solid_angle.eps')
plt.savefig(plot_path, format='eps')
plt.show()

### 2.3. Compute window view horizontal angle on each grid point

In [ ]:
grid_df["horizontal_angle"] = 360 * gb['hw_mask'].sum() / gb['horizon_mask'].sum()
grid_df

In [ ]:
plt.figure(figsize = (6,8))
ha_array = grid_df["horizontal_angle"].to_numpy().reshape((-1, 25))
plt.imshow(ha_array, interpolation='none')

plt.axis('off')
hamx = np.floor(ha_array.max())
hamn = np.ceil(ha_array.min())
tcks = np.arange(0, 360, 30).astype(np.int)
cbar = plt.colorbar(ticks=[hamn] + list(tcks[(tcks > hamn) * (tcks < hamx)]) + [hamx]) 
cbar.ax.get_yaxis().labelpad = 20
cbar.ax.set_ylabel('Window Horizontal Angle (Degree)', rotation=270, ha='center', va='center')          

plt.yticks(rotation = 45)
# cbar.ax.set_yticklabels([1]*10, rotation='vertical')
plt.setp(cbar.ax.get_yticklabels(),rotation=270)
plot_path = os.path.join('plots', project_name + '_horizontal_angle.png')
plt.savefig(plot_path, facecolor="#FFFFFF", dpi=3, dpi=1200)
plt.show()

### 2.4. Compute view layer count on each grid point

In [ ]:
counts_df = gb['last_int_name'].value_counts()

count_dict = counts_df.to_dict()
new_dict = {}
for key, val in count_dict.items():
    if key[0] in new_dict.keys():
        new_dict[key[0]][key[1]] = val
    else:
        new_dict[key[0]] = {key[1]:val}

view_layer_count_df = pd.DataFrame.from_dict(new_dict).T.fillna(0)
grid_df = pd.concat([grid_df, view_layer_count_df], axis = 1)
grid_df

In [ ]:
columns = list(view_layer_count_df.columns)
fig, axs = plt.subplots(1, len(columns), figsize=(15,6))
fig.tight_layout() 
for ax, column in zip(axs,columns):
    count_grid = np.reshape(view_layer_count_df[column].to_numpy() - 1, (-1, 25))
    pcm =ax.pcolormesh(np.flip(count_grid))
    ax.axis('off')
    ax.set_title(column)
    fig.colorbar(pcm, ax=ax, orientation="vertical", pad=0.05)

plot_path = os.path.join('plots', project_name + '_all_layers.eps')
plt.savefig(plot_path, format='eps')
# cbar = fig.colorbar()              
# cbar.set_ticks([1,2,3,4])

### 2.5. View layer count with different thresholds

In [ ]:
through_win_ratio = view_layer_count_df.to_numpy() / view_layer_count_df.to_numpy().sum(axis=1).reshape(-1, 1)
through_win_ratio

In [ ]:
through_win_ratio_wo_wall = through_win_ratio[:, 1:]

In [ ]:
import matplotlib.gridspec as gridspec

plt.style.use(['ggplot'])

div = 800.0
plot_count = 5
w_ratio = 10
fig3 = plt.figure(constrained_layout=True, figsize=(20, 8))
fig3.patch.set_facecolor('#FFFFFF')
gs = fig3.add_gridspec(1, w_ratio * plot_count + 1)
cmap = plt.get_cmap('viridis', 4)
for i in range(plot_count):
    ax = fig3.add_subplot(gs[:, i*w_ratio:(i+1)*w_ratio])
    ax.set_title('t = ' + ('%.3f' % ((0.001 + i/div) * 4 * np.pi)) + ' sr', color='black')
    ax.axis('off')
    count_grid = np.reshape((through_win_ratio_wo_wall> (0.001 + i/div)).sum(axis=1), (-1, 25))
    im = ax.imshow(count_grid, interpolation='none', cmap=cmap,vmin = -.5, vmax = 3+.5)

cax = fig3.add_subplot(gs[:, w_ratio * plot_count ])
cb = plt.colorbar(im, ticks=(), cax = cax) 
for i in np.arange(0,4):
    cb.ax.text(0.75, i - 0.05, i)

plot_path = os.path.join('plots', project_name + '_view_layer_threshold.png')
plt.savefig(plot_path, facecolor="#FFFFFF", dpi=300)


In [ ]:
def discrete_matshow(data):
    # get discrete colormap
    cmap = plt.get_cmap('viridis', np.max(data)-np.min(data)+1)
    # set limits .5 outside true range
    mat = plt.imshow(data, cmap=cmap,vmin = np.min(data)-.5, vmax = np.max(data)+.5)
    #tell the colorbar to tick at integers
    cax = plt.colorbar(mat, ticks=np.arange(np.min(data),np.max(data)+1))
    plt.axis('off')
    cax.ax.set_ylabel('t = ' + ('%.3f' % ((0.001 + 4/div) * 4 * np.pi)) + ' sr', color='black', rotation=270, ha='center', va='center')  
    cax.ax.get_yaxis().labelpad = 10
    plt.setp(cbar.ax.get_yticklabels(),rotation=270)

discrete_matshow(count_grid)

In [ ]:
filepath = os.path.join(src, "data", "ref_room_window.obj")
win_mesh = pv.read(filepath)

In [ ]:
win_normal = np.array(win_mesh.face_normals).flatten()

In [ ]:
win_mask = grid_results["window_mask"].to_numpy()
distances = grid_results[["first_int_dist",	"last_int_dist"]].to_numpy()
ray_dir = grid_results[["Rx","Ry","Rz"]].to_numpy()

In [ ]:
win_mask = (grid_results["last_int_name"] != "wall").to_numpy()
not_sky_mask = (grid_results["last_int_name"] != "window").to_numpy()

In [ ]:
win_dist = distances[win_mask]
win_ray_dir = ray_dir[win_mask]

In [ ]:
r = np.sum(win_normal * win_ray_dir, axis=1)
dist_dif = (win_dist[:, 0] - win_dist[:, 1]) 

In [ ]:
dist = dist_dif * r

In [ ]:
con = 

In [ ]:
nosky_dist_finall = dist[(dist_dif > 100) * (dist_dif < 400)]


In [ ]:
nosky_dist_finall.max()